# Opgave:

Datasæt: https://www.opendata.dk/city-of-copenhagen/monumenter

CSV: https://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:monumenter&outputFormat=csv&SRSNAME=EPSG:4326

1. Hvor mange monumenter er der i København?

2. Hvor mange monumenter bliver vedligeholdt? Dvs. graffitirenhold = ja

3. Lav en funktion som kan finde koordinaterne på et monument baseret på monumentets id eller navn?
	F.eks:
		def monumentById(monumentId):
			return coordinates
	
    - 3A) Vis monument som bliver returneret i metoden, på et kort over københavn ved brug af plotting. (Se afsnittet om 'Folium and Bokeh' under notebooks/03-3 Plotting)
 

4. Find navnet på monumentet med x og y koordinaterne eller længde- og breddegraderne?
	
	x = 724407.424966
	y = 6175719.798486
	
	MULTIPOINT ((12.555485308174104 55.69383926601615))

5. Lav en metode der optegner alle monumenterne på kortet ved brug af plotting.

6. Gør kortet interaktiv så navnet på monumenterne vises når man trykker på et plot. (Se afsnittet om 'Interactive plots with bokeh' under notebooks/03-3 Plotting)

##### Minimusløsning: pkt 3 + 3a + 5

In [131]:
# importing everything we should need for the assingment

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

data = pd.read_csv('CSV_files/monumenter.CSV') # read the csv file.

# check to see if we got the file
data

,FID,id,navn,loebenummer,web_id,x,y,longitude,latitude,graffitirenhold,wkb_geometry
0,monumenter.1,49690.0,Efterår,115.0,167.0,724712.263534,6.179443e+06,12.577001,55.708568,Ja,MULTIPOINT ((12.577000674229012 55.70856778229...
1,monumenter.2,49865.0,H.W. Meyer,41.0,93.0,723888.874104,6.178082e+06,12.562810,55.696741,Ja,MULTIPOINT ((12.562810238950165 55.69674138769...
2,monumenter.3,49866.0,Sigurd Fafnersbane og slange,314.0,353.0,724293.902081,6.178228e+06,12.569361,55.697869,Ja,MULTIPOINT ((12.569361482261764 55.69786887177...
3,monumenter.4,50058.0,Satyr med bacchusbarn,21.0,67.0,724122.061749,6.176324e+06,12.565077,55.680871,Ja,MULTIPOINT ((12.56507667919525 55.680870776003...
4,monumenter.5,50063.0,Talerstol,290.0,330.0,724157.041343,6.178457e+06,12.567375,55.699982,Ja,MULTIPOINT ((12.567375194374895 55.69998215768...
...,...,...,...,...,...,...,...,...,...,...,...
384,monumenter.385,61658.0,Siddende kvinde,149.0,198.0,725581.180764,6.175222e+06,12.587330,55.670313,Ja,MULTIPOINT ((12.587330246990874 55.67031312733...
385,monumenter.386,NaN,NaN,NaN,NaN,727108.232799,6.174361e+06,12.610848,55.661876,NaN,MULTIPOINT ((12.610848146663862 55.66187624531...
386,monumenter.387,61724.0,Modstandsbevægelsens faldne,160.0,206.0,726086.812842,6.175080e+06,12.595237,55.668805,Ja,MULTIPOINT ((12.59523651729718 55.668805127961...
387,monumenter.388,65076.0,Kongens Livjægere,161.0,207.0,725524.513763,6.178248e+06,12.588918,55.697472,Ja,MULTIPOINT ((12.588918117651739 55.69747240261...


## 1. Hvor mange monumenter er der i København?

In [132]:
monumenter_Amount = data['FID']

monumenter_Amount.count()

389

## 2. Hvor mange monumenter bliver vedligeholdt? Dvs. graffitirenhold = ja

In [133]:
monumenter_vedligeholdt = data['graffitirenhold']

monumenter_vedligeholdt.count()

363

## 3. Lav en funktion som kan finde koordinaterne på et monument baseret på monumentets id eller navn? F.eks:

3. Lav en funktion som kan finde koordinaterne på et monument baseret på monumentets id eller navn?
	F.eks:
		def monumentById(monumentId):
			return coordinates
 

In [143]:
def monumentById(monumentId):
    
    monuments = data[['id','navn','x','y','longitude','latitude']].sort_values('id',ascending=True)
    monuments['id'] = range(1,len(monuments)+1)
    series = monuments.loc[monuments['id'] == monumentId, ['id','navn','x','y','longitude','latitude']]
    
    # could make the output more pretty when using this method,
    # by returning a string with some text explaining what monument was found
    cordinates = series
    
    return cordinates


In [159]:
monumentById(6)

,id,navn,x,y,longitude,latitude
5,6,Ole Syversen,724015.981189,6.178451e+06,12.565131,55.699997


## 3.A) Vis monument som bliver returneret i metoden, på et kort over københavn ved brug af plotting.
- (Se afsnittet om 'Folium and Bokeh' under notebooks/03-3 Plotting)

In [145]:
import json
import requests
import folium
import urllib

def get_city_location(city='Copenhagen'):
    """Get the location coordinates from OpenStreetMaps"""
    url_nomatim_api = 'https://nominatim.openstreetmap.org/search'
    r = requests.get(url_nomatim_api, params={'format': 'json', 'city': city})
    results = r.json()  # Potentially many matches
    print('results: ',results)
    location = results[0]
    lat, lon = float(location['lat']), float(location['lon'])
    return lat, lon


def get_geojson(url): 
    """
    return the response body as json
    GeoJSON is an open standard format designed for representing simple geographical features, along with their non-spatial attributes
    """
    response = requests.get(url)
    geo_json = response.json()
    return geo_json


cph_lat, cph_lon = get_city_location()
url_environ = 'http://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:miljozone&outputFormat=json&SRSNAME=EPSG:4326'
# url_environ = 'https://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:monumenter&outputFormat=json&SRSNAME=EPSG:4326'
geo_json = get_geojson(url_environ) # contains attribute: geometri: type: multipolygon
print('\n\nGeoJSON open standard format:\n',geo_json)

results:  [{'place_id': 283423199, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 13707878, 'boundingbox': ['55.5267243', '55.8467243', '12.4100724', '12.7300724'], 'lat': '55.6867243', 'lon': '12.5700724', 'display_name': 'København, 1357, Danmark', 'class': 'place', 'type': 'city', 'importance': 0.7199038383120139, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_place_city.p.20.png'}, {'place_id': 281985125, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 175928, 'boundingbox': ['43.881475', '43.905213', '-75.686912', '-75.659037'], 'lat': '43.893439', 'lon': '-75.673828', 'display_name': 'Copenhagen, Denmark, Lewis County, New York, United States', 'class': 'boundary', 'type': 'administrative', 'importance': 0.5150655424601236, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.20.png'}, {'place_

In [161]:
cordinates = monumentById(7)

map_osm = folium.Map(location=(cph_lat, cph_lon), zoom_start=10) # show the map
for i in range(0,len(cordinates)):
    folium.Marker(
        location=[cordinates.iloc[i]['latitude'],cordinates.iloc[i]['longitude']],
        popup=cordinates.iloc[i]['navn'],
        ).add_to(map_osm)
map_osm.save('./osm2.html')
map_osm

## 5. Lav en metode der optegner alle monumenterne på kortet ved brug af plotting.

In [156]:
# draw all the monuments


def allMonumentPoints():
    
    for i in range(0,len(data)):
        folium.Marker(
        location=[data.iloc[i]['latitude'],data.iloc[i]['longitude']],
        popup=data.iloc[i]['navn'],
        ).add_to(map_osm)
        
allMonumentPoints()
map_osm.save('./osm2.html')
map_osm